##### Fixing the seeds during your experimentation stage is important in order to be sure that your improvements are real and not due to randomness. 

##### In this problem I just wanted to give a try to the LightGBM model and I bumped into the following reproducibility issue (that drove me crazy):

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import random
import os
import numpy as np

# Function to fix some seeds
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

SEED = 42
seed_everything(SEED)

# Read train data
df_train = pd.read_csv("/kaggle/input/tabular-playground-series-dec-2021/train.csv")
df_train.drop(["Id"], axis=1, inplace=True)

# Only sample a smaller fraction of the train data for speed purposes
df_train = df_train.sample(frac=0.25, random_state=SEED)

I used a simple train-test split because it is only to show the no reproducible results:

In [ ]:
y = df_train["Cover_Type"]
X = df_train.drop(["Cover_Type"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

Ok, now let's fit the model, predict on the test set, and calculate the accuracy score. Let's do it 3 times and check whether the scores are different:

In [ ]:
params = {
    "objective": "multiclass",
    "num_classes": 7,
    "random_state": SEED,
}

for _ in range(3):
    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train)
    y_test_pred = model.predict(X_test)
    print(accuracy_score(y_test, y_test_pred))

You (probably) see that the values are not the same, and the gap between those is significant (I said probably because sometimes some of them are the same!). 

Now we can try to repeat the same process for binary classification. First, convert the target to binary:

In [ ]:
y = (df_train["Cover_Type"] > 3).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [ ]:
params = {
    "objective": "binary",
    "num_classes": 1,
    "random_state": SEED,
}

for _ in range(3):
    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train)
    y_test_pred = model.predict(X_test)
    print(accuracy_score(y_test, y_test_pred))

For binary classification the accuracy scores are the same. What happens with the multiclass problem? Is it doing something under the hood that I'm not aware of?